In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

import json
import os
import random

import torch
import torch.nn as nn

import torchvision.models as models
import torchvision.transforms as transforms


%matplotlib inline

In [ ]:
torch.manual_seed(3001)
import random
# random.seed(3001)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Load pretrained VGG16 model for transfer learning:

In [ ]:
# model = models.vgg16(pretrained=False)

Freeze all feature extraction layers:

In [ ]:
# for param in model.features.parameters():
#     param.requires_grad = False

Replace the output layer with a single node:

In [ ]:
# num_ftrs = model.classifier[6].in_features
# model.classifier[6] = nn.Linear(num_ftrs, 1)

Train the model:

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def load_model(checkpoint_path: str, device):
    model = models.vgg16(pretrained=False)
    
    # Freeze all feature extraction layers:
    for param in model.features.parameters():
        param.requires_grad = False
    
    # Replace the output layer with a single node:
    num_ftrs = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(num_ftrs, 1)
    
    # Move model to gpu if avaliable:
    model.to(device=device, dtype = torch.float32)
    
    # Apply checkpoint
    if os.path.isdir(checkpoint_path):
        checkpoint = torch.load(os.path.join(checkpoint_path, 'model.pt'), map_location=device)
        model.load_state_dict(checkpoint['model'])
    #     optimizer.load_state_dict(checkpoint['optimizer'])
        model.eval()
        print("Model retrieved succesfully.")
        return model
    else:
        print("Model could not be found")
        return None

In [ ]:
# move model to gpu if avaliable:
# model.to(device=device, dtype = torch.float32)

# if os.path.isdir('../input/image-rotation-prediction-training'):
#     checkpoint = torch.load('../input/image-rotation-prediction-training/model.pt', map_location=device)
#     model.load_state_dict(checkpoint['model'])
# #     optimizer.load_state_dict(checkpoint['optimizer'])
#     model.eval()
#     print("Model retrieved succesfully.")
# else:
#     print("Model could not be found")
# model = load_model('../input/image-rotation-prediction-training', device)
model = load_model('../input/imagerotationexternaltrainingcheckpoint', device)

In [ ]:
import PIL

def rotate_upright(img: PIL.Image, model, device):
    img_tensor = transforms.ToTensor()(img).unsqueeze_(0)
    img_tensor = img_tensor.to(device=device, dtype = torch.float32)

    rotation_prediction = model(img_tensor).squeeze()
    
    corrected_image = img.rotate(-rotation_prediction)
    return corrected_image, rotation_prediction

In [ ]:
import torchvision.transforms.functional as TF

class RotateUprightTransform:
    
    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = load_model('../input/imagerotationexternaltrainingcheckpoint', self.device)
    
    def load_model(checkpoint_path: str, device):
        model = models.vgg16(pretrained=False)

        # Freeze all feature extraction layers:
        for param in model.features.parameters():
            param.requires_grad = False

        # Replace the output layer with a single node:
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs, 1)

        # Move model to gpu if avaliable:
        model.to(device=device, dtype = torch.float32)

        # Apply checkpoint
        if os.path.isdir(checkpoint_path):
            checkpoint = torch.load(os.path.join(checkpoint_path, 'model.pt'), map_location=device)
            model.load_state_dict(checkpoint['model'])
        #     optimizer.load_state_dict(checkpoint['optimizer'])
            model.eval()
            print("Model retrieved succesfully.")
            return model
        else:
            print("Model could not be found")
            return None
    
    def __call__(self, img: PIL.Image, return_angle=False):
        img_tensor = transforms.ToTensor()(img).unsqueeze_(0)
        img_tensor = img_tensor.to(device=self.device, dtype = torch.float32)

        rotation_prediction = self.model(img_tensor).squeeze().item()

        corrected_image = TF.rotate(img, rotation_prediction)#img.rotate(-rotation_prediction)
        if return_angle:
            return corrected_image, rotation_prediction
        return corrected_image

In [ ]:
import PIL

def rotate_and_crop(image: PIL.Image, angle_deg: int):
    w_i,h_i = image.size
    
    rotated_expanded_image_with_padding = image.rotate(-angle_deg, expand=True)
    w_p,h_p = rotated_expanded_image_with_padding.size
    
    import math
    angle_rad = np.deg2rad(abs(angle_deg))
    rem_deg_rad = np.deg2rad(90-(abs(angle_deg) % 90))
    
    w_to_remove = int(math.cos(rem_deg_rad)*h_i)
    h_to_remove = int(math.sin(angle_rad)*w_i)
    return rotated_expanded_image_with_padding.crop((w_to_remove,h_to_remove,w_p-w_to_remove,h_p-h_to_remove))

def randomly_rotate_image(image: PIL.Image):
    def do_nothing(image: PIL.Image):
        return image, 0

    def rotate_with_max_degrees(image: PIL.Image, max_degrees: int = 10):
        rotation_deg = random.randrange(-abs(max_degrees), abs(max_degrees) + 1)
        return (rotate_and_crop(image, rotation_deg), rotation_deg)

    def rotate_about_90_degrees_left(image: PIL.Image):
        rotated_90_degrees_left = image.rotate(90, expand=True)
        rotated_image, further_degrees = rotate_with_max_degrees(rotated_90_degrees_left, 5)
        return (rotated_image, -90 + further_degrees)

    def rotate_about_90_degrees_right(image: PIL.Image):
        rotated_90_degrees_right = image.rotate(-90, expand=True)
        rotated_image, further_degrees = rotate_with_max_degrees(rotated_90_degrees_right, 5)
        return (rotated_image, 90 + further_degrees)

    random_image_rotation_fun = np.random.choice(
        [
            do_nothing,
            rotate_with_max_degrees,
            rotate_about_90_degrees_left,
            rotate_about_90_degrees_right
        ],
        p=[
            0.6,
            0.2,
            0.1,
            0.1
        ]
    )

    return random_image_rotation_fun(image)

In [ ]:
import PIL
input_path = r'../input/hotel-id-to-combat-human-trafficking-2022-fgvc9'
train_image_path = os.path.join(input_path, r"train_images")

random_hotel_images_path = os.path.join(train_image_path, random.choice(os.listdir(train_image_path)))
# random_hotel_image_path = '../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/11260/000039269.jpg'
random_hotel_image_path = os.path.join(random_hotel_images_path, random.choice(os.listdir(random_hotel_images_path)))
print("Random hotel folder: {}".format(random_hotel_image_path))
random_hotel_image = PIL.Image.open(random_hotel_image_path)
print("Original image dimensions: {}".format(random_hotel_image.size))
angle_deg = 8

rotated_random_hotel_image = rotate_and_crop(random_hotel_image, angle_deg) # Temporary, for testing purposes

rotate_upright_transform = RotateUprightTransform()
corrected_image, hotel_pred = rotate_upright_transform(rotated_random_hotel_image, True)#rotate_upright(rotated_random_hotel_image, model, device)
print("Prediction: {}".format(hotel_pred))

import matplotlib.pyplot as plt
f, axarr = plt.subplots(1,3)
axarr[0].imshow(random_hotel_image)
axarr[1].imshow(rotated_random_hotel_image)
axarr[2].imshow(corrected_image)